In [5]:
import  numpy as np
import  pandas as pd

In [11]:
Xy=pd.read_csv("../Data/EditedData.csv.gz")

In [64]:
Xy

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,0.375699,-0.187453,-0.370271,-0.241215,0.241293,-0.174012,0.523549,-0.167292,-0.242571,-0.221054,...,0.188892,0.274902,0.666045,-0.403865,0.612271,-0.097654,0.519489,-0.101444,0.760120,238 236 237 238 240 240 239 241 241 243 240 23...
1,0.340270,-0.271457,-0.376057,-0.303152,0.226170,-0.265118,0.473390,-0.246101,-0.249277,-0.284135,...,0.159603,0.175447,0.590667,-0.268284,0.584326,-0.027404,0.463887,-0.052752,0.780837,219 215 204 196 204 211 212 200 180 168 178 19...
2,0.355355,-0.272716,-0.356171,-0.272716,0.237750,-0.243313,0.478842,-0.243313,-0.215039,-0.243313,...,0.115395,0.267145,0.521132,-0.297368,0.515250,-0.015105,0.462329,-0.015105,0.638737,144 142 159 180 188 188 184 180 167 132 84 59 ...
3,0.358870,-0.223713,-0.332852,-0.223713,0.250070,-0.184850,0.506557,-0.200397,-0.216278,-0.192623,...,0.128470,0.366643,0.514661,-0.224052,0.545739,0.047983,0.460243,0.074191,0.630591,193 192 193 194 194 194 193 192 168 111 50 12 ...
4,0.390110,-0.174557,-0.328233,-0.207458,0.220123,-0.174557,0.510748,-0.169074,-0.229534,-0.185524,...,0.351865,0.263988,0.615067,-0.350172,0.604110,-0.063276,0.535571,-0.078601,0.809816,147 148 160 196 215 214 216 217 219 220 206 18...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4275,0.307501,-0.272460,-0.399591,-0.253826,0.179076,-0.237709,0.479741,-0.242746,-0.236416,-0.229148,...,0.251362,0.261533,0.541974,-0.341617,0.526008,-0.014763,0.599184,-0.011659,0.611159,21 21 18 19 33 34 48 60 61 70 89 99 99 100 102...
4276,0.471301,-0.222683,-0.369260,-0.243729,0.301020,-0.206101,0.667092,-0.206739,-0.219388,-0.213754,...,0.301516,0.416346,0.690661,-0.394336,0.650605,-0.016083,0.704516,-0.026818,0.726482,9 7 3 5 10 15 17 19 23 23 19 11 7 5 6 5 4 5 5 ...
4277,0.398003,-0.216312,-0.425643,-0.194751,0.215310,-0.185682,0.549549,-0.185510,-0.284570,-0.164565,...,0.318533,0.541375,0.611854,-0.434675,0.614496,-0.024596,0.646732,0.017151,0.865511,197 198 198 197 195 191 184 155 71 43 33 37 38...
4278,0.437481,-0.148582,-0.336504,-0.360592,0.276122,-0.156105,0.648954,-0.074795,-0.177201,-0.289022,...,0.244496,0.192143,0.680923,-0.536057,0.464039,-0.209226,0.652113,-0.221064,0.674074,38 20 9 9 40 20 34 41 27 30 18 13 5 6 5 4 5 7 ...
